<a href="https://colab.research.google.com/github/mynhungg/Datamining/blob/Anh-Ki%E1%BB%87t/PhoBERT_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

path = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(path)

In [ ]:
!git clone https://github.com/sLeeNguyen/nlp-text-classification.git

Cloning into 'nlp-text-classification'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 4), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), 162.50 KiB | 652.00 KiB/s, done.


In [ ]:
%cd nlp-text-classification

/content/drive/MyDrive/Colab Notebooks/nlp-text-classification


In [ ]:
!chmod +x prepare.sh
!./prepare.sh

--2023-05-26 06:59:54--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.35.166.28, 13.35.166.24, 13.35.166.17, ...
Connecting to public.vinai.io (public.vinai.io)|13.35.166.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  44.7MB/s    in 42s     

2023-05-26 07:00:36 (28.4 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt
--2023-05-26 07:01:20--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connec

Lệnh **`chmod +x prepare.sh`** được sử dụng để cấp quyền thực thi cho tệp tin **`prepare.sh`**. Điều này cho phép chạy tệp tin shell **`prepare.sh`** như một chương trình thực thi.

Lệnh **`./prepare.sh`** được sử dụng để thực thi tệp tin shell **`prepare.sh`**. Trong trường hợp này, khi chạy lệnh này, nó sẽ thực thi các câu lệnh được định nghĩa trong tệp tin **`prepare.sh`**.

In [ ]:
!pip3 install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Lệnh **`!pip install -r requirements.txt`** được sử dụng để cài đặt các gói phụ thuộc từ tệp tin **`requirements.txt`**.

In [ ]:
import torch
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

from fairseq.models.roberta import RobertaModel, RobertaHubInterface
from fairseq.data import Dictionary
from fairseq.data.encoders.fastbpe import fastBPE

from vncorenlp import VnCoreNLP

- **`import torch`**: Đây là thư viện được sử dụng cho việc tính toán số học và xây dựng mạng nơ-ron trong PyTorch.
- **`import numpy as np`**: Đây là thư viện cho tính toán số học và xử lý mảng nhiều chiều trong Python.
- **`import pickle`**: Đây là thư viện cho việc lưu trữ và truy xuất dữ liệu dưới dạng đối tượng Python bằng cách sử dụng giao thức pickle.
- **`from sklearn.preprocessing import LabelEncoder`**: Đây là một lớp trong thư viện scikit-learn được sử dụng để mã hóa nhãn văn bản thành dạng số.
-** `from fairseq.models.roberta import RobertaModel, RobertaHubInterface`**: Đây là các lớp và giao diện được cung cấp bởi fairseq, một thư viện sử dụng trong nghiên cứu và triển khai mô hình ngôn ngữ.
- **`from fairseq.data import Dictionary`**: Đây là một lớp từ thư viện fairseq để xử lý từ điển văn bản.
- **`from fairseq.data.encoders.fastbpe import fastBPE`**: Đây là một lớp từ fairseq để mã hóa văn bản bằng phương pháp mã hóa nhanh BPE (Byte Pair Encoding).
- **`from vncorenlp import VnCoreNLP`**: Đây là một gói thư viện NLP cho tiếng Việt được gọi là VnCoreNLP, được sử dụng để xử lý văn bản tiếng Việt.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

`torch.device('cuda' if torch.cuda.is_available() else 'cpu')` tạo một đối tượng thiết bị (device) trong PyTorch, với giá trị là `'cuda'` nếu có sẵn GPU hỗ trợ CUDA, ngược lại là `'cpu'`.

In [ ]:
class TextClassifier():
    def __init__(self) -> None:
        self.max_seq_len = 256
        self.__load_model()
        self.__load_vocab()
        self.__load_label_encoder()
        self.__rdrsegmenter = VnCoreNLP(
            "vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size="-Xmx500m")

    def get_model(self) -> RobertaModel:
        return self.__model

    def __load_model(self):
        print('Loading NLP model...')
        self.__model = RobertaModel.from_pretrained(
            'PhoBERT_base_fairseq', checkpoint_file='model.pt')
        self.__model.to(DEVICE)
        self.__model.register_classification_head('new_task', num_classes=10)
        self.__model.load_state_dict(torch.load(
          '/content/drive/MyDrive/Colab Notebooks/VNTC/model_ckpt_full_both/model.bin', 
          map_location=DEVICE))

        class BPE():
            bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
        self.__model.bpe = self.__bpe = fastBPE(BPE())
        print('\nModel has loaded successfully.')

    def __load_vocab(self):
        print('\nLoading dictionary...')
        self.__vocab = Dictionary()
        self.__vocab.add_from_file('PhoBERT_base_fairseq/dict.txt')
        print('\nDictionary has loaded')

    def __load_label_encoder(self):
        with open('/content/drive/MyDrive/Colab Notebooks/VNTC/labelEncoder.pkl', 'rb') as f:
            self.le: LabelEncoder = pickle.load(f)

    def encode(self, text) -> torch.LongTensor:
        text_tokenized = ' '.join([
            ' '.join(sent) for sent in self.__rdrsegmenter.tokenize(text)
        ])
        bpe_sentence = '<s> ' + self.__bpe.encode(text_tokenized) + ' <s>'
        tokens = self.__vocab.encode_line(
            bpe_sentence, append_eos=False, add_if_not_exist=False)
        # Index cac token cls (đầu câu), eos (cuối câu), padding (padding token)
        # cls_id = 0
        eos_id = 2
        pad_id = 1
        if len(tokens) > self.max_seq_len:
            tokens = tokens[:self.max_seq_len]
            tokens[-1] = eos_id
        else:
            tokens = torch.cat((tokens, torch.tensor(
                [pad_id, ] * (self.max_seq_len - len(tokens)))))
        return tokens.long()

    def predict(self, text):
        self.__model.eval()
        tokens = self.encode(text)
        logits = self.__model.predict('new_task', tokens, return_logits=True)
        y_pred = np.argmax(logits.detach().cpu().numpy(), axis=1)
        return self.le.inverse_transform(y_pred)[0]

Lớp **`TextClassifier`** là một lớp đóng gói các chức năng liên quan đến phân loại văn bản. Dưới đây là giải thích về từng phương thức và thuộc tính trong lớp này:

- **`__init__(self)`**: Phương thức khởi tạo của lớp. Nó thiết lập giá trị cho các thuộc tính **`max_seq_len`**, và gọi các phương thức **`__load_model()`**, **`__load_vocab()`**, **`__load_label_encoder()`** để tải các thành phần cần thiết.

- **`get_model(self) -> RobertaModel`**: Phương thức trả về mô hình **`RobertaModel`** được sử dụng bên trong lớp.

- **`__load_model(self)`**: Phương thức tải mô hình **`RobertaModel`** từ tệp checkpoint đã được huấn luyện trước và tải trọng số lưu trữ trong tệp **`model.bin**`. Nó cũng cấu hình mô hình cho tác vụ phân loại mới với số lớp là 10.

- **`__load_vocab(self)`**: Phương thức tải từ điển từ vựng từ tệp `dict.txt`.

- **`__load_label_encoder(self)`**: Phương thức tải trình mã hóa nhãn từ tệp **`labelEncoder.pkl`** bằng cách sử dụng **`pickle`**.

- **`encode(self, text) -> torch.LongTensor`**: Phương thức mã hóa một đoạn văn bản thành một tensor **`torch.LongTensor`** sử dụng từ điển từ vựng. Nó chia văn bản thành các câu và mã hóa chúng bằng phương pháp mã hóa BPE. Đoạn văn bản được mã hóa sẽ có chiều dài tối đa là **`max_seq_len`**.

- **`predict(self, text)`**: Phương thức dự đoán nhãn cho một đoạn văn bản. Nó mã hóa đoạn văn bản và sử dụng mô hình để dự đoán nhãn. Cuối cùng, nó chuyển đổi nhãn dự đoán từ dạng số sang dạng chuỗi bằng cách sử dụng **`LabelEncoder`**.

In [ ]:
classifier = TextClassifier()

Loading NLP model...


1042301B [00:00, 52656821.04B/s]
456318B [00:00, 7218206.83B/s]



Model has loaded successfully.

Loading dictionary...

Dictionary has loaded


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
text = '''
       Sau này có rất nhiều lần trời đổ mưa, nhưng anh đã không đội mưa đến gặp em thêm một lần nào nữa.
       
       '''

In [ ]:
print(classifier.predict(text))

Doi song
